In [256]:
epochs = 30
n_train_items = 12800

In [250]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [251]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]


workers = connect_to_workers(n_workers=20)


In [258]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [257]:
# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ]))
#     .federate(workers), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

less_train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
less_test_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(test_loader)
        if i < n_train_items / args.batch_size
    ]




In [6]:
# from PIL import Image
# import numpy 
# #mnist_dataset.__getitem__(2)[1]
# a = (mnist_dataset.__getitem__(0)[0]).numpy()
# a.dtype = 'uint8'
# print(a)
# Image.fromarray(a[0], mode= 'P')

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [17]:
def model_init(workers, Net):
    model_list = list()
    for worker in workers:
        model_list.append(Net)
    return model_list
def opt_init(model_list):
    opt_list = list()
    for model  in model_list:
        opt_list.append(optim.SGD(model.parameters(), lr=args.lr))
    return opt_list

In [239]:
def train(args, device, less_train_dataloader, opt_list, epoch, workers):
    global model_list
    ## start training and record the model into model_list
    
    for batch_idx, (data, target) in enumerate(less_train_dataloader): # <-- now it is a distributed dataset
        model_on_worker = model_list[batch_idx%len(workers)]
        model_on_worker.train()
        model_on_worker.send(workers[batch_idx%len(workers)]) # <-- NEW: send the model to the right location
        
        data_on_worker = data.send(workers[batch_idx%len(workers)])
        target_on_worker = target.send(workers[batch_idx%len(workers)])
        
        data_on_worker, target_on_worker = data_on_worker.to(device), target_on_worker.to(device)
        
        opt_list[batch_idx%len(workers)].zero_grad()
        
        output = model_on_worker(data_on_worker)
        loss = F.nll_loss(output, target_on_worker)
        loss.backward()
        
        opt_list[batch_idx%len(workers)].step()
        model_on_worker.get() # <-- NEW: get the model back
        
        model_list[batch_idx%len(workers)] = model_on_worker #When len(dataloader) is longer than the len(worker) send and get must be modified
        #model_list here is full of the model which has trained on the workers, there are all different now.
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(less_train_dataloader) * args.batch_size,
                100. * batch_idx / len(less_train_dataloader), loss.item()))


    ##Aggregation time
    new_model = []
    tmp_model = Net().to(device)
    with torch.no_grad():
        for p in model_list[0].parameters():
            new_model.append(0)
            
        for m in model_list:
            for par_idx, par in enumerate(m.parameters()):
                #average the model_list
                new_model[par_idx] = new_model[par_idx]+par.data
                # we get new model in list format and need to set_ to model
        for worker in range(len(workers)):
            for par_idx in range(len(new_model)):
                list(model_list[worker].parameters())[par_idx].set_(new_model[par_idx]/len(workers))
        #init model with new_model
        


In [240]:
for i,j in enumerate([5,5]):
    print(i)

0
1


In [241]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader*args.batch_size)
    #Since the test loader here is a list, we can get the len by * it with batch.size

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader* args.batch_size),
        100. * correct / (len(test_loader)*args.batch_size)))

In [259]:
%%time
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

model_list = []
model_list = model_init(workers, Net().to(device))
opt_list = opt_init(model_list)
# not finish in train, finish latter
pars = [list(model.parameters()) for model in model_list]

for epoch in range(1, args.epochs + 1):
    train(args, device, less_train_dataloader, opt_list, epoch, workers)
    print("After training")
    test(args, model_list[0], device, less_test_dataloader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/12800 (0%)]	Loss: 2.288257
Train Epoch: 1 [1920/12800 (15%)]	Loss: 2.178536
Train Epoch: 1 [3840/12800 (30%)]	Loss: 1.928699
Train Epoch: 1 [5760/12800 (45%)]	Loss: 1.526911
Train Epoch: 1 [7680/12800 (60%)]	Loss: 0.991821
Train Epoch: 1 [9600/12800 (75%)]	Loss: 0.653335
Train Epoch: 1 [11520/12800 (90%)]	Loss: 0.655600
After training

Test set: Average loss: 0.5874, Accuracy: 8152/10048 (81%)

Train Epoch: 2 [0/12800 (0%)]	Loss: 0.582307
Train Epoch: 2 [1920/12800 (15%)]	Loss: 0.349787
Train Epoch: 2 [3840/12800 (30%)]	Loss: 0.244426
Train Epoch: 2 [5760/12800 (45%)]	Loss: 0.367783
Train Epoch: 2 [7680/12800 (60%)]	Loss: 0.369737
Train Epoch: 2 [9600/12800 (75%)]	Loss: 0.296277
Train Epoch: 2 [11520/12800 (90%)]	Loss: 0.511333
After training

Test set: Average loss: 0.3229, Accuracy: 8989/10048 (89%)

Train Epoch: 3 [0/12800 (0%)]	Loss: 0.308202
Train Epoch: 3 [1920/12800 (15%)]	Loss: 0.194156
Train Epoch: 3 [3840/12800 (30%)]	Loss: 0.151043
Train Epoch: 3 [5760/1280